In [1]:
#I.input data and drop id
import numpy as np
import pandas as pd
import os
trainx = pd.read_csv('../original_data/task2/X_train.csv')
trainy = pd.read_csv('../original_data/task2/y_train.csv')
testx  = pd.read_csv('../original_data/task2/X_test.csv')

xdropid=trainx.drop('id', axis=1)
ydropid= trainy.y
testx_noid= testx.drop('id', axis=1)
print(xdropid.shape,ydropid.shape,testx_noid.shape)

#2. standardlize and add new outlier variable
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
print(scaler.fit(xdropid))

x_scale= pd.DataFrame(scaler.transform(xdropid))
print(x_scale.head())
testx_scale=pd.DataFrame(scaler.transform(testx_noid))
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

rng = np.random.RandomState(42)

clf = IsolationForest(behaviour='new',random_state=rng, contamination=0.1)
clf.fit(x_scale)
x_clf = clf.predict(x_scale)
testx_clf= clf.predict(testx_scale)
print((x_clf<0).sum())
print((testx_clf<0).sum())
x_clf=pd.DataFrame(x_clf)
x_add=pd.DataFrame(np.concatenate((x_scale, x_clf), axis=1))
print(x_add.shape)

testx_clf=pd.DataFrame(testx_clf)
testx_add=pd.DataFrame(np.concatenate((testx_scale, testx_clf), axis=1))
print(testx_add.shape)

(4800, 1000) (4800,) (4100, 1000)
RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)
        0         1         2         3         4         5         6    \
0 -1.814205 -0.563601  0.357447 -1.917004  1.466177 -1.021554 -2.842974   
1  0.216027  0.141576 -0.625123 -0.091898 -0.014292  0.064482  0.392830   
2  0.223604  0.223606  0.554346  0.138732 -0.453637 -0.375028  0.251915   
3 -0.489692 -0.055772  0.825809 -0.253400  0.857272 -0.631391 -0.345909   
4  0.305676  0.108558  0.022180 -0.387695 -0.411647  0.189445  0.603706   

        7         8         9      ...          990       991       992  \
0 -0.465324 -0.009342 -0.692873    ...    -1.743932 -1.941933 -1.232187   
1  0.210195 -0.078851 -0.421578    ...     0.083989  0.061831  0.387153   
2  0.672453  0.890189 -0.495019    ...     0.286262  0.103720 -0.391375   
3 -0.679789  1.067118 -0.101326    ...    -1.203255 -0.611784 -1.215225   
4  0.003878 -0.728250 -0.219178    ...  

In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [9]:
# 3. select some features, the results are not bad without feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
skb= SelectKBest(f_classif, k=600)
skb.fit(x_add,ydropid)
X_new= skb.transform(x_add)
testx_new= skb.transform(testx_add)
print(X_new.shape)

(4800, 600)


In [10]:
# 4. rbf svc cross validation 
from sklearn.svm import SVC
clf = SVC(C=1, cache_size=200, coef0=0.0,
    decision_function_shape='ovr', degree=5, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True,
    tol=1e-5, class_weight='balanced')
print(cross_val_score(clf, X_new, ydropid,scoring='balanced_accuracy', cv=5))

[0.69398148 0.70555556 0.71018519 0.69537037 0.70231481]


In [5]:
# 5. predict and get output
newy=clf.fit(X_new, ydropid).predict(testx_new)
print(newy)


output= pd.DataFrame({'id': testx.id, 'y': newy})
output.to_csv('../../output6.csv', index = False)

[0. 0. 1. ... 1. 1. 1.]
